In [ ]:
# 기본 import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 분석기 설치 및 import
! pip install kss konlpy # 한국어분석기 kss, konlpy 설치
import nltk, kss, konlpy # nltk 영어분석기 kss, konlpy한국어분석
nltk.__version__  ,  konlpy.__version__ # 설치 확인 print(분석기.__버전__)

# 토크나이저 import 및 실행
from tensorflow.keras.preprocessing.text import text_to_word_sequence # 한국어 단어 토크나이저 : 단어 기준 토큰화 : 잘라주는
from nltk.tokenize import word_tokenize, WordPunctTokenizer, TreebankWordTokenizer  # nltk 영어 단어 토크나이저 
from nltk.tokenize import sent_tokenize # nltk 영어 [문장] 토크나이저
nltk.download('punkt') # nltk 리소스 설치
nltk.download('averaged_perceptron_tagger') # nltk 리소스 설치

# (필요시) 토크나이저로 쪼갠 후 태그 분석
from konlpy.tag import Okt, Kkma # Okt를 쓰자 # 한국어태그 분석기 [클래스]  # 형태소 분석 후 형태소를 기준으로 잘라냄 # Okt사용법 : # Okt().pos(한국어)  # 한국어 단어 기준 품사 태깅 하기, 단어토크나이즈+태그 기능 #  Okt().morphs(한국어) : Okt로 한국어 단어 토큰화 하기  # Okt().nouns(한국어) : 명사(진짜명사)만 뽑겟다  # Kkma 는 Okt 와 사용법이 같다 기본적으로 범용적인 [Okt를 쓰자] 
from nltk.tag import  pos_tag # 영어태그 분석기 # 포스태그실행법 : pos_tag(토크나이저실행문) # 태깅하기

# 문장토큰화
from nltk.tokenize import sent_tokenize # nltk 영어 [문장] 토크나이저
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secre The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
문장토큰화_된_data = sent_tokenize(text) #문장토크나이저로 문장토큰화 문장별로 자르기 # 3개의 문장 

# 불용성단어 선언
from nltk.corpus import stopwords 
nltk.download('stopwords') # 불용어모음집 다운로드
불용성단어 = set(stopwords.words('english')) #영어로구성된불용어모음집가져오기  # 중복안되게 set 으로 받았다  #거를 단어

# 단어토크화, 불용성단어처리, 소문자화, 짧은단어처리
단어모음 = {}
t_data = []
for 문장 in 문장토큰화_된_data:        # 1 문장토큰화한 문장을 하나씩 뽑는다
    단어토큰화_된_data = word_tokenize(문장) # 2 문장을 단어 토크나이저에 넣어서 단어별로 쪼갠다
    l=[]  # 첫 문장의 단어들이 다 돌고 새 문장의 단어들이 들어올 때 리스트를 자꾸 새로 선언하는 것이다.
    for 단어 in 단어토큰화_된_data:     # 3 단어토큰화된 데이터에서 단어를 하나씩 뽑는다
        소문자화된_단어 = 단어.lower()  # 4 단어를 하나씩 뽑아서 소문자로 바꾼다  # 영어는 소문자로 통합 필수
        if 소문자화된_단어 not in 불용성단어:  #  5  불용성단어집에 없는지 확인한다  불용성단어가 아니라면
            if len(소문자화된_단어) > 2 :  # 6 단어의 길이가 3이상이라면 (알파벳짧은 것 의미 적을 거니까 배제해보자 2보다 큰 것을 담는다)
                l.append(소문자화된_단어)   # 7 l리스트에 넣고
                if 소문자화된_단어 not in 단어모음: # 그 단어가 단어모음에 없다면
                    단어모음[소문자화된_단어]= 0  #  8 단어모음 딕셔너리에 그 단어를 key로 해서 밸류는 0을 넣는다 ( 키만 설정해주는 것이다)
                단어모음[소문자화된_단어]+= 1    # 9 밸류를 +1해준다  ( 원래 단어모음에 있었다면 키가 이미 있으니까 여기서 +1만 해준다 )
    t_data.append(l)  # 첫 줄에서 한 문장이 들어가서 단어별로 쪼개서 소문자화 하고 불용성단어가 아니며 3이상의 길이라면 어펜드 > 초기data가 문장3개라 리스트3개가 들어왔다
t_data, 단어모음  #단어모음은 출현 수를 기록한 것이다 # 토큰화 끝 : 결과적으로 t_data 는 소문자화, 불용어처리, 단어토큰화까지 3개를 동시에 처리한 데이터 (+짧은단어처리) # 정수인코딩 필요 : 빈도 수에 따른 숫자로 바꿔줘야함 # 패딩 필요 : 입력렝스 맞추기

# 패딩시 인풋 길이 '맥스렌값' 찾기 : 인풋 길이를 담은 어레이를 만들고 중간값을 알아낸 후 패딩 맥스렌으로 일률적으로 인풋 길이를 맞춘다.
l = np.array( [len(x) for x in t_data ] ) # 각 인풋 묶음의 길이 값이 모였다
plt.hist(l) # 길이값의 편향을 확인하여 맥스렌 값을 결정, 대칭형태면 평균과 중앙값이 비슷하다. 편향이 심할수록 중앙값이 좋다, 실질적인 데이터는 편향이 심한 것이 많다
np.median(l) # 중앙값이 무조건 좋은 것은 아니다. 시각화를 통해 데이터를 적게 잃고 0 를 줄이는 값을 찾는 것이 좋다. 평균: np.mean(l), 최대: np.max(l) # median 미디언의 의미 : 최대최소의 중간값이 아니라 리스트에서 딱 중앙에 있는 걸 뜻한다. ex) 4,5,50,1000,2000 이면 중앙값은 50 이고 평균은 600 이다
 
# t_data(정수인코딩끝난데이터)를 패딩
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tk = Tokenizer(num_words = 6) # (num_words = 살릴단어수+2 ,oov_token='oov' ) / (num_words = 살릴단어수+1 ) # 여기서 자동으로 특수문자제거, 소문자화 가 디폴트  # (oov_token=True) : 아웃오브보캐를 넣을지, 꼭True라는 문자가 아니어도 된다 'oov' 아웃오브보캐가 많으면 빈도가 높으니 1번으로 지정 받는다  
tk.fit_on_texts(t_data)  # t_data 기준 핏
print(tk.index_word) # 빈도순 정렬이다 1이 가장 빈도가 높다 
t_x = tk.texts_to_sequences(t_data)   # 텍스트투시퀀스: 시퀀스 숫자로 변경
ck_t_x = pad_sequences(t_x, maxlen=3)  # 생략시 가장 긴 길이로 0 값을 자동으로 강제로 늘려서 학습에 방해되고 짧게 적용시 데이터손실이 있으니 적정값을 찾자 보통 중앙값 활용
print(t_x)  # 정수인코딩 완료한 데이터 (num_words= 숫자 에서 -1 위까지 살린다)
print(ck_t_x) # 패딩 완료한 데이터 